## First I will download all the dependencies that we will need

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported successfully.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

## Download and Export Dataset

As stated the dataset has already been located as a .json file so I will download the dataset using wget command.

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data loaded successfully')

Data loaded successfully


Now, I will load the data and explore the data

In [3]:
with open('newyork_data.json') as json_data:
    newyorkcity_data = json.load(json_data)

In [4]:
newyorkcity_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

We can notice that all data is in features key, so I will define a new key that contains the features variable only.

In [5]:
newyork_features=newyorkcity_data['features']
newyork_features[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

## Tranforming this data into a pandas dataframe

I will tranform this data into a pandas dataframe. For that step I will first create an empty dataframe and then load the newyork_features data into it.

In [6]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
newyork_boroughs = pd.DataFrame(columns=column_names)
newyork_boroughs

,Borough,Neighborhood,Latitude,Longitude


Lets load newyork_features data in this dataframe now

In [7]:
for data in newyork_features:
    borough = data['properties']['borough'] 
    newyork_name = data['properties']['name']
        
    newyork_latlon = data['geometry']['coordinates']
    newyork_lat = newyork_latlon[1]
    newyork_lon = newyork_latlon[0]
    
    newyork_boroughs = newyork_boroughs.append({'Borough': borough,
                                          'Neighborhood': newyork_name,
                                          'Latitude': newyork_lat ,
                                          'Longitude': newyork_lon}, ignore_index=True)

In [8]:
newyork_boroughs.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


We have to make sure that the dataframe contains five boroughs

In [9]:
print('The dataframe has {} boroughs'.format(
        len(newyork_boroughs['Borough'].unique())
    )
)

The dataframe has 5 boroughs


## Using Geopy Library to get Longitude and Latitude of New York City

In [10]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


## Creating a map of New York using Folium library

In [11]:
# create map of New York using latitude and longitude values
newyork_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(newyork_boroughs['Latitude'], newyork_boroughs['Longitude'], newyork_boroughs['Borough'], newyork_boroughs['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(newyork_map)  
    
newyork_map

The above map shows the boroughs and neighborhoods of New York City. We will use this data to compare the amount of "Indian Stores" and "NightClubs" in Manhattan, Queens and Brooklyn.

## I. Exploring Manhattan

I will start with exploring Manhattan and create a new dataframe manhattan_df where I will slice the original dataframe newyork_boroughs

In [12]:
manhattan_df = newyork_boroughs[newyork_boroughs['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


### Getting geographical coordinates of Manhattan

In [13]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


### Visualizing the map of Manhattan 

In [14]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_df['Latitude'], manhattan_df['Longitude'], manhattan_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

## Loading Foursqure Credentials

In [15]:
CLIENT_ID = '2KIL3JFRYX4SZMR5NKS1W0AYWYBCY3S2PZPTDLTUXRA5I041' # Foursquare ID
CLIENT_SECRET = 'HYFYZVEHV5IUQPUA4KBS2ZTNQV0JZ1EP2XDODFMHGGQRV10W' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2KIL3JFRYX4SZMR5NKS1W0AYWYBCY3S2PZPTDLTUXRA5I041
CLIENT_SECRET:HYFYZVEHV5IUQPUA4KBS2ZTNQV0JZ1EP2XDODFMHGGQRV10W


I have obtained a random address from Google Maps and will use it to conduct the analysis in Manhattan

In [16]:
address = '116 Macdougal St ,New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.72968975 -74.00046838292411


### I will run a query for Indian Stores in the area first

In [17]:
search_query = 'Indian'
radius = 1000
print(search_query + ' .... OK!')

Indian .... OK!


Define the corresponding URL

In [18]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius)
url

'https://api.foursquare.com/v2/venues/search?client_id=2KIL3JFRYX4SZMR5NKS1W0AYWYBCY3S2PZPTDLTUXRA5I041&client_secret=HYFYZVEHV5IUQPUA4KBS2ZTNQV0JZ1EP2XDODFMHGGQRV10W&ll=40.72968975,-74.00046838292411&v=20180605&query=Indian&radius=1000'

Sednding Get request to obtain and examine the result

In [19]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5edb6400b1cac0001b59f070'},
 'response': {'venues': [{'id': '4116be80f964a520f90b1fe3',
    'name': 'Panna II Garden Indian Restaurant',
    'location': {'address': '93 1st Ave',
     'crossStreet': 'btwn 5th & 6th St',
     'lat': 40.726272990045985,
     'lng': -73.98627301759028,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.726272990045985,
       'lng': -73.98627301759028},
      {'label': 'entrance', 'lat': 40.726254, 'lng': -73.986488}],
     'distance': 1256,
     'postalCode': '10003',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['93 1st Ave (btwn 5th & 6th St)',
      'New York, NY 10003',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d10f941735',
      'name': 'Indian Restaurant',
      'pluralName': 'Indian Restaurants',
      'shortName': 'Indian',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/indian_',

In [20]:
# assign relevant part of JSON to venues
manhattan_venues = results['response']['venues']

# tranform venues into a dataframe
venues_manhattan_df = json_normalize(manhattan_venues)
venues_manhattan_df.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,4116be80f964a520f90b1fe3,Panna II Garden Indian Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1591436402,False,93 1st Ave,btwn 5th & 6th St,40.726273,-73.986273,"[{'label': 'display', 'lat': 40.72627299004598...",1256,10003,US,New York,NY,United States,"[93 1st Ave (btwn 5th & 6th St), New York, NY ...",149505937,NaN,NaN,NaN,NaN,NaN,NaN
1,5b17c9f935d3fc002c2c09db,"Indian Egg Donors DGA, Inc.","[{'id': '54541900498ea6ccd0202697', 'name': 'H...",v-1591436402,False,1148 5th Ave,Suite 1B,40.728007,-73.997040,"[{'label': 'display', 'lat': 40.72800677563629...",344,10128,US,New York,NY,United States,"[1148 5th Ave (Suite 1B), New York, NY 10128, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4d992c1e942ba093ebb5968c,THELEwala,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1591436402,False,112 Macdougal St,at Minetta St.,40.729724,-74.000730,"[{'label': 'display', 'lat': 40.72972369809264...",22,10012,US,New York,NY,United States,"[112 Macdougal St (at Minetta St.), New York, ...",NaN,286721,https://www.seamless.com/menu/thelewala-112-ma...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png
3,4e4e6a0cbd4101d0d7a9a45a,Sangam Authentic Indian Cuisine,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1591436402,False,190 Bleecker St,NaN,40.728915,-74.003229,"[{'label': 'display', 'lat': 40.72891535176595...",248,10012,US,New York,NY,United States,"[190 Bleecker St, New York, NY 10012, United S...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5df11a0d80d83900088ffbcc,Indian Biryani House,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1591436402,False,NaN,NaN,40.728253,-74.007321,"[{'label': 'display', 'lat': 40.728253, 'lng':...",599,10014,US,New York,NY,United States,"[New York, NY 10014, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now I will filter the Dataframe to only show the information that is important to the client

In [21]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in venues_manhattan_df.columns if col.startswith('location.')] + ['id']
venues_manhattan_df_filtered = venues_manhattan_df.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
venues_manhattan_df_filtered['categories'] = venues_manhattan_df_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
venues_manhattan_df_filtered.columns = [column.split('.')[-1] for column in venues_manhattan_df_filtered.columns]

venues_manhattan_df_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Panna II Garden Indian Restaurant,Indian Restaurant,93 1st Ave,btwn 5th & 6th St,40.726273,-73.986273,"[{'label': 'display', 'lat': 40.72627299004598...",1256,10003,US,New York,NY,United States,"[93 1st Ave (btwn 5th & 6th St), New York, NY ...",4116be80f964a520f90b1fe3
1,"Indian Egg Donors DGA, Inc.",Health & Beauty Service,1148 5th Ave,Suite 1B,40.728007,-73.997040,"[{'label': 'display', 'lat': 40.72800677563629...",344,10128,US,New York,NY,United States,"[1148 5th Ave (Suite 1B), New York, NY 10128, ...",5b17c9f935d3fc002c2c09db
2,THELEwala,Indian Restaurant,112 Macdougal St,at Minetta St.,40.729724,-74.000730,"[{'label': 'display', 'lat': 40.72972369809264...",22,10012,US,New York,NY,United States,"[112 Macdougal St (at Minetta St.), New York, ...",4d992c1e942ba093ebb5968c
3,Sangam Authentic Indian Cuisine,Indian Restaurant,190 Bleecker St,NaN,40.728915,-74.003229,"[{'label': 'display', 'lat': 40.72891535176595...",248,10012,US,New York,NY,United States,"[190 Bleecker St, New York, NY 10012, United S...",4e4e6a0cbd4101d0d7a9a45a
4,Indian Biryani House,Food Truck,NaN,NaN,40.728253,-74.007321,"[{'label': 'display', 'lat': 40.728253, 'lng':...",599,10014,US,New York,NY,United States,"[New York, NY 10014, United States]",5df11a0d80d83900088ffbcc
5,Indian Curry Mahal,Indian Restaurant,78 2nd Ave,NaN,40.726337,-73.988930,"[{'label': 'display', 'lat': 40.726337, 'lng':...",1042,10003,US,New York,NY,United States,"[78 2nd Ave, New York, NY 10003, United States]",4e4c4f0bbd413c4cc66864bf
6,Indian Restaurant,Indian Restaurant,369 Broome St,NaN,40.720272,-73.995855,"[{'label': 'display', 'lat': 40.72027216898879...",1118,10013,US,New York,NY,United States,"[369 Broome St, New York, NY 10013, United Sta...",4d0e6159bf0c8eec2188282a
7,Karahi Indian Cuisine,Food,508 Broome St,NaN,40.723621,-74.003124,"[{'label': 'display', 'lat': 40.723621, 'lng':...",711,10013,US,New York,NY,United States,"[508 Broome St, New York, NY 10013, United Sta...",4f32b59819836c91c7f207fc
8,Crazy Indian Person's Party,None,200 Mott St.,NaN,40.721995,-73.995919,"[{'label': 'display', 'lat': 40.721995, 'lng':...",938,NaN,US,NaN,New York,United States,"[200 Mott St., New York, United States]",4e45e718c65bd6ffbe8b4ac7
9,Naveli Indian Cuisine,Indian Restaurant,NaN,NaN,40.727135,-73.988944,"[{'label': 'display', 'lat': 40.72713464404035...",1012,10003,US,New York,NY,United States,"[New York, NY 10003, United States]",501f00cde4b0ccb681ff5700


In [22]:
venues_manhattan_df_filtered.count()

name                24
categories          23
address             22
crossStreet         13
lat                 24
lng                 24
labeledLatLngs      24
distance            24
postalCode          23
cc                  24
city                23
state               24
country             24
formattedAddress    24
id                  24
dtype: int64

 ### We can observe that there are 24 Indian Stores in this area. 
 We will check the rating of two randomly selected restaurants to get some idea of the quality of the food as well. 

In [23]:
venue_id = '3fd66200f964a52021e31ee3' # ID of Haveli
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/3fd66200f964a52021e31ee3?client_id=2KIL3JFRYX4SZMR5NKS1W0AYWYBCY3S2PZPTDLTUXRA5I041&client_secret=HYFYZVEHV5IUQPUA4KBS2ZTNQV0JZ1EP2XDODFMHGGQRV10W&v=20180605'

### Send GET request for result

In [24]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'venuePage', 'reasons', 'page', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'popular', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '3fd66200f964a52021e31ee3',
 'name': 'Haveli',
 'contact': {'phone': '2129820533', 'formattedPhone': '(212) 982-0533'},
 'location': {'address': '100 2nd Ave',
  'crossStreet': 'at 5th St',
  'lat': 40.72720942119707,
  'lng': -73.98859836084183,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.72720942119707,
    'lng': -73.98859836084183}],
  'postalCode': '10003',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['100 2nd Ave (at 5th St)',
   'New York, NY 10003',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/haveli/3fd66200f964a52021e31ee3',
 'categories': [{'id': '4bf58dd8d48988d10f941735',
   'name': 'Indian Restaurant',
   'pluralName': 'Indian Restaurants',
   'shortName': 'Indian',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/indian_',
    'suffix': '.png'},
   'primary': True}],
 'verified': True,
 'stats': {'tipCount': 26},
 'url': 'http://havelinyc.us',
 'price': {

In [25]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.0


This rating for this restaurant is alright. Now we shall check rating for another restaurant.

In [26]:
venue_id = '4a3812dbf964a520af9e1fe3' # ID of Raj Mahal
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')


7.1


The Rating for the second restaurant is slightly better then the first one.

### Now I shall run a query for Nightclubs in the area

I shall use the address I obtained from Google Maps to run this query as well

In [27]:
address = '116 Macdougal St ,New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.72968975 -74.00046838292411


We shall increase the radius for Nightclubs

In [28]:
search_query = 'Nightclub'
radius = 5000
print(search_query + ' .... OK!')

Nightclub .... OK!


In [54]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius)
url

'https://api.foursquare.com/v2/venues/search?client_id=2KIL3JFRYX4SZMR5NKS1W0AYWYBCY3S2PZPTDLTUXRA5I041&client_secret=HYFYZVEHV5IUQPUA4KBS2ZTNQV0JZ1EP2XDODFMHGGQRV10W&ll=40.7566484,-73.9140858&v=20180605&query=Nightclub&radius=5000'

In [30]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5edb64277828ae001b2c83f8'},
 'response': {'venues': [{'id': '4fbedc1de4b06efb316f1435',
    'name': 'Bonfire Nightclub',
    'location': {'address': '404 Avenue of the Americas',
     'crossStreet': 'W 8th Street',
     'lat': 40.73374801031597,
     'lng': -73.99958834964578,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.73374801031597,
       'lng': -73.99958834964578}],
     'distance': 457,
     'postalCode': '10011',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['404 Avenue of the Americas (W 8th Street)',
      'New York, NY 10011',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d11f941735',
      'name': 'Nightclub',
      'pluralName': 'Nightclubs',
      'shortName': 'Nightclub',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/nightclub_',
       'suffix': '.png'},
      'primary': True}],
    'referralId'

In [31]:
# assign relevant part of JSON to venues
manhattan_venues = results['response']['venues']

# tranform venues into a dataframe
venues_manhattan_df = json_normalize(manhattan_venues)
venues_manhattan_df.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,4fbedc1de4b06efb316f1435,Bonfire Nightclub,"[{'id': '4bf58dd8d48988d11f941735', 'name': 'N...",v-1591436417,False,404 Avenue of the Americas,W 8th Street,40.733748,-73.999588,"[{'label': 'display', 'lat': 40.73374801031597...",457,10011,US,New York,NY,United States,"[404 Avenue of the Americas (W 8th Street), Ne...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4ec861cacc21b428e42e21b2,J'adore Nightclub,"[{'id': '4bf58dd8d48988d11f941735', 'name': 'N...",v-1591436417,False,37 W 17th St,Bet 5th And 6th Ave.,40.738887,-73.994243,"[{'label': 'display', 'lat': 40.73888687204467...",1150,10011,US,New York,NY,United States,"[37 W 17th St (Bet 5th And 6th Ave.), New York...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5caf985d666116002ca0eab5,VANDAL Nightclub,"[{'id': '4bf58dd8d48988d11f941735', 'name': 'N...",v-1591436417,False,199 Bowery Lowr Level,NaN,40.721201,-73.993455,"[{'label': 'display', 'lat': 40.72120070108200...",1114,10002,US,New York,NY,United States,"[199 Bowery Lowr Level, New York, NY 10002, Un...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5a96e751d69ed035676ca89a,Mission Nightclub,"[{'id': '4bf58dd8d48988d11f941735', 'name': 'N...",v-1591436417,False,229 W 28th St,Between 7th & 8th Ave,40.747934,-73.994962,"[{'label': 'display', 'lat': 40.74793425253044...",2083,10001,US,New York,NY,United States,"[229 W 28th St (Between 7th & 8th Ave), New Yo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5d7dcb1004b75b00085b977e,Truth Nightclub (Ghost Club Power),"[{'id': '4bf58dd8d48988d11f941735', 'name': 'N...",v-1591436417,False,441 W 14th St,15th st,40.742066,-74.006872,"[{'label': 'display', 'lat': 40.74206599999999...",1479,10014,US,New York,NY,United States,"[441 W 14th St (15th st), New York, NY 10014, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now I will filter the Dataframe to only show the information that is important to the client

In [32]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in venues_manhattan_df.columns if col.startswith('location.')] + ['id']
venues_manhattan_df_filtered = venues_manhattan_df.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
venues_manhattan_df_filtered['categories'] = venues_manhattan_df_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
venues_manhattan_df_filtered.columns = [column.split('.')[-1] for column in venues_manhattan_df_filtered.columns]

venues_manhattan_df_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Bonfire Nightclub,Nightclub,404 Avenue of the Americas,W 8th Street,40.733748,-73.999588,"[{'label': 'display', 'lat': 40.73374801031597...",457,10011,US,New York,NY,United States,"[404 Avenue of the Americas (W 8th Street), Ne...",NaN,4fbedc1de4b06efb316f1435
1,J'adore Nightclub,Nightclub,37 W 17th St,Bet 5th And 6th Ave.,40.738887,-73.994243,"[{'label': 'display', 'lat': 40.73888687204467...",1150,10011,US,New York,NY,United States,"[37 W 17th St (Bet 5th And 6th Ave.), New York...",NaN,4ec861cacc21b428e42e21b2
2,VANDAL Nightclub,Nightclub,199 Bowery Lowr Level,NaN,40.721201,-73.993455,"[{'label': 'display', 'lat': 40.72120070108200...",1114,10002,US,New York,NY,United States,"[199 Bowery Lowr Level, New York, NY 10002, Un...",NaN,5caf985d666116002ca0eab5
3,Mission Nightclub,Nightclub,229 W 28th St,Between 7th & 8th Ave,40.747934,-73.994962,"[{'label': 'display', 'lat': 40.74793425253044...",2083,10001,US,New York,NY,United States,"[229 W 28th St (Between 7th & 8th Ave), New Yo...",NaN,5a96e751d69ed035676ca89a
4,Truth Nightclub (Ghost Club Power),Nightclub,441 W 14th St,15th st,40.742066,-74.006872,"[{'label': 'display', 'lat': 40.74206599999999...",1479,10014,US,New York,NY,United States,"[441 W 14th St (15th st), New York, NY 10014, ...",NaN,5d7dcb1004b75b00085b977e
5,1 OAK,Nightclub,453 W 17th St,btwn 9th & 10th Ave,40.743838,-74.006357,"[{'label': 'display', 'lat': 40.74383758799102...",1651,10011,US,New York,NY,United States,"[453 W 17th St (btwn 9th & 10th Ave), New York...",NaN,4b73ab13f964a520dbb72de3
6,Blue Nightclub,Nightclub,220 W 44th St,NaN,40.757651,-73.987150,"[{'label': 'display', 'lat': 40.757651, 'lng':...",3309,10036,US,New York,NY,United States,"[220 W 44th St, New York, NY 10036, United Sta...",NaN,5cfc88ce3fffb4002572fa4e
7,Butter Night Club,Nightclub,NaN,NaN,40.728718,-73.992570,"[{'label': 'display', 'lat': 40.72871812122733...",674,10003,US,New York,NY,United States,"[New York, NY 10003, United States]",NaN,4e0027482271bc79ee0659bf
8,Avenue,Nightclub,116 10th Ave,btwn 17th & 18th St,40.744114,-74.006642,"[{'label': 'display', 'lat': 40.74411375620596...",1687,10011,US,New York,NY,United States,"[116 10th Ave (btwn 17th & 18th St), New York,...",Chelsea,4ad9f641f964a5206e1c21e3
9,Marquee,Nightclub,289 10th Ave,btwn W 26th & W 27th St,40.750097,-74.002814,"[{'label': 'display', 'lat': 40.75009748526309...",2280,10001,US,New York,NY,United States,"[289 10th Ave (btwn W 26th & W 27th St), New Y...",NaN,40a2ba80f964a520fef21ee3


In [33]:
venues_manhattan_df_filtered.count()

name                30
categories          30
address             25
crossStreet         19
lat                 30
lng                 30
labeledLatLngs      30
distance            30
postalCode          27
cc                  30
city                29
state               30
country             30
formattedAddress    30
neighborhood         1
id                  30
dtype: int64

### There are 30 Nightclub venues near the address we used.

## II. Exploring Queens

I shall create a new dataframe queens_df from the original dataframe newyork_boroughs

In [34]:
queens_df = newyork_boroughs[newyork_boroughs['Borough'] == 'Queens'].reset_index(drop=True)
queens_df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138


### Getting geographical coordinates of Queens

In [35]:
address = 'Queens, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Queens are 40.7498243, -73.7976337.


### Visualizing the map of Queens

In [36]:
# create map of Queens using latitude and longitude values
map_queens = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(queens_df['Latitude'], queens_df['Longitude'], queens_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_queens)  
    
map_queens

I have obtained a random address from Google Maps and will use it to conduct the analysis in Queens

### Loading Foursquare Credentials

In [37]:
CLIENT_ID = '2KIL3JFRYX4SZMR5NKS1W0AYWYBCY3S2PZPTDLTUXRA5I041' # Foursquare ID
CLIENT_SECRET = 'HYFYZVEHV5IUQPUA4KBS2ZTNQV0JZ1EP2XDODFMHGGQRV10W' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2KIL3JFRYX4SZMR5NKS1W0AYWYBCY3S2PZPTDLTUXRA5I041
CLIENT_SECRET:HYFYZVEHV5IUQPUA4KBS2ZTNQV0JZ1EP2XDODFMHGGQRV10W


In [38]:
address = '46th St Sunnyside, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7566484 -73.9140858


### I will run a query for Indian Stores in the area first

In [39]:
search_query = 'Indian'
radius = 1000
print(search_query + ' .... OK!')

Indian .... OK!


Defining the Corresponding url

In [40]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius)
url

'https://api.foursquare.com/v2/venues/search?client_id=2KIL3JFRYX4SZMR5NKS1W0AYWYBCY3S2PZPTDLTUXRA5I041&client_secret=HYFYZVEHV5IUQPUA4KBS2ZTNQV0JZ1EP2XDODFMHGGQRV10W&ll=40.7566484,-73.9140858&v=20180605&query=Indian&radius=1000'

In [41]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5edb63e4fb34b5001b2ca758'},
 'response': {'venues': [{'id': '50e5c5a2e4b035f761f9df9f',
    'name': 'Saffron Indian Cuisine',
    'location': {'address': '44-04 Broadway',
     'crossStreet': 'at 44th St',
     'lat': 40.75740906705734,
     'lng': -73.91562587277978,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.75740906705734,
       'lng': -73.91562587277978}],
     'distance': 155,
     'postalCode': '11103',
     'cc': 'US',
     'city': 'Astoria',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['44-04 Broadway (at 44th St)',
      'Astoria, NY 11103',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d10f941735',
      'name': 'Indian Restaurant',
      'pluralName': 'Indian Restaurants',
      'shortName': 'Indian',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/indian_',
       'suffix': '.png'},
      'primary': True}],
    'delivery': {'id': '274958',
   

In [42]:
# assign relevant part of JSON to venues
queens_venues = results['response']['venues']

# tranform venues into a dataframe
venues_queens_df = json_normalize(queens_venues)
venues_queens_df.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id
0,50e5c5a2e4b035f761f9df9f,Saffron Indian Cuisine,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1591436435,False,44-04 Broadway,at 44th St,40.757409,-73.915626,"[{'label': 'display', 'lat': 40.75740906705734...",155,11103,US,Astoria,NY,United States,"[44-04 Broadway (at 44th St), Astoria, NY 1110...",274958,https://www.seamless.com/menu/saffron-indian-c...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,53805381
1,538b922b498e7ba2b290a647,Delicious Indian Food Cart,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1591436435,False,NaN,NaN,40.758493,-73.919255,"[{'label': 'display', 'lat': 40.758493, 'lng':...",481,NaN,US,Queens,NY,United States,"[Queens, NY, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4aa56c81f964a5204e4820e3,Seva Indian Cuisine,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1591436435,False,30-07 34th St,btwn 30th & 31st Ave.,40.765521,-73.919157,"[{'label': 'display', 'lat': 40.76552091157529...",1076,11103,US,Astoria,NY,United States,"[30-07 34th St (btwn 30th & 31st Ave.), Astori...",71405,https://www.seamless.com/menu/seva-indian-cuis...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,94541120
3,59b9d25d0d173f3bdf654d77,Tikka Indian Grill,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1591436435,False,34-14 30th Ave,35th street,40.765397,-73.918658,"[{'label': 'display', 'lat': 40.765397, 'lng':...",1047,11103,US,New York,NY,United States,"[34-14 30th Ave (35th street), New York, NY 11...",345013,https://www.seamless.com/menu/tikka-indian-gri...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,NaN
4,4c48da9f3013a59356c5f0e1,Saffron Garden,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1591436435,False,46-11 Skillman Ave,btwn 46th and 47th,40.746554,-73.917738,"[{'label': 'display', 'lat': 40.74655397129589...",1165,11104,US,Sunnyside,NY,United States,"[46-11 Skillman Ave (btwn 46th and 47th), Sunn...",436933,https://www.seamless.com/menu/saffron-garden-4...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,NaN


Now I will filter the Dataframe to only show the information that is importaant to the client

In [43]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in venues_queens_df.columns if col.startswith('location.')] + ['id']
venues_queens_df_filtered = venues_queens_df.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
venues_queens_df_filtered['categories'] = venues_queens_df_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
venues_queens_df_filtered.columns = [column.split('.')[-1] for column in venues_queens_df_filtered.columns]

venues_queens_df_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Saffron Indian Cuisine,Indian Restaurant,44-04 Broadway,at 44th St,40.757409,-73.915626,"[{'label': 'display', 'lat': 40.75740906705734...",155,11103,US,Astoria,NY,United States,"[44-04 Broadway (at 44th St), Astoria, NY 1110...",50e5c5a2e4b035f761f9df9f
1,Delicious Indian Food Cart,Food Truck,NaN,NaN,40.758493,-73.919255,"[{'label': 'display', 'lat': 40.758493, 'lng':...",481,NaN,US,Queens,NY,United States,"[Queens, NY, United States]",538b922b498e7ba2b290a647
2,Seva Indian Cuisine,Indian Restaurant,30-07 34th St,btwn 30th & 31st Ave.,40.765521,-73.919157,"[{'label': 'display', 'lat': 40.76552091157529...",1076,11103,US,Astoria,NY,United States,"[30-07 34th St (btwn 30th & 31st Ave.), Astori...",4aa56c81f964a5204e4820e3
3,Tikka Indian Grill,Indian Restaurant,34-14 30th Ave,35th street,40.765397,-73.918658,"[{'label': 'display', 'lat': 40.765397, 'lng':...",1047,11103,US,New York,NY,United States,"[34-14 30th Ave (35th street), New York, NY 11...",59b9d25d0d173f3bdf654d77
4,Saffron Garden,Indian Restaurant,46-11 Skillman Ave,btwn 46th and 47th,40.746554,-73.917738,"[{'label': 'display', 'lat': 40.74655397129589...",1165,11104,US,Sunnyside,NY,United States,"[46-11 Skillman Ave (btwn 46th and 47th), Sunn...",4c48da9f3013a59356c5f0e1


In [44]:
venues_queens_df_filtered.count()

name                5
categories          5
address             4
crossStreet         4
lat                 5
lng                 5
labeledLatLngs      5
distance            5
postalCode          4
cc                  5
city                5
state               5
country             5
formattedAddress    5
id                  5
dtype: int64

 ### We can observe that there are only 5 Indian Stores in this area. 
 We will check the rating of two randomly selected restaurants to get some idea of the quality of the food as well. 

In [45]:
venue_id = '50e5c5a2e4b035f761f9df9f' # ID of Saffron Indian Cuisine
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.2


The rating is better alright. Now let's check the rating of another restaurant

In [46]:
venue_id = '4aa56c81f964a5204e4820e3' # ID of Seva Indian Cuisine
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

9.0


The rating for this restaurant is very high so we can recommend it to our client even if he does not moved to Queens

### Now I shall run a query for Nightclubs in the area

I shall use the address I obtained from Google Maps to run this query as well

In [47]:
address = '46th St Sunnyside, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7566484 -73.9140858


We shall increase the radius for Nightclubs

In [48]:
search_query = 'Nightclub'
radius = 5000
print(search_query + ' .... OK!')

Nightclub .... OK!


In [49]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius)
url

'https://api.foursquare.com/v2/venues/search?client_id=2KIL3JFRYX4SZMR5NKS1W0AYWYBCY3S2PZPTDLTUXRA5I041&client_secret=HYFYZVEHV5IUQPUA4KBS2ZTNQV0JZ1EP2XDODFMHGGQRV10W&ll=40.7566484,-73.9140858&v=20180605&query=Nightclub&radius=5000'

In [50]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5edb64fcb9a389001b50e6d5'},
 'response': {'venues': [{'id': '4bc92319511f9521eb4aadc7',
    'name': 'PureNY Nightclub',
    'location': {'address': '62-05 30th Ave',
     'lat': 40.76005082320337,
     'lng': -73.90125017948878,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.76005082320337,
       'lng': -73.90125017948878}],
     'distance': 1146,
     'postalCode': '11377',
     'cc': 'US',
     'city': 'Woodside',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['62-05 30th Ave',
      'Woodside, NY 11377',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d11f941735',
      'name': 'Nightclub',
      'pluralName': 'Nightclubs',
      'shortName': 'Nightclub',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/nightclub_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1591436704',
    'hasPerk': False},
   {'id': '42a39200f964a520ea241fe

In [51]:
# assign relevant part of JSON to venues
queens_venues = results['response']['venues']

# tranform venues into a dataframe
venues_queens_df = json_normalize(queens_venues)
venues_queens_df.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood,venuePage.id,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,4bc92319511f9521eb4aadc7,PureNY Nightclub,"[{'id': '4bf58dd8d48988d11f941735', 'name': 'N...",v-1591436704,False,62-05 30th Ave,40.760051,-73.901250,"[{'label': 'display', 'lat': 40.76005082320337...",1146,11377,US,Woodside,NY,United States,"[62-05 30th Ave, Woodside, NY 11377, United St...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,42a39200f964a520ea241fe3,Star Nightclub,"[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",v-1591436704,False,246 Columbus Ave,40.777270,-73.979207,"[{'label': 'display', 'lat': 40.77727, 'lng': ...",5950,10023,US,New York,NY,United States,"[246 Columbus Ave (btw 71st and 72nd), New Yor...",btw 71st and 72nd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5a58a0b5c47cf954cad00ad1,Doha Nightclub NYC,"[{'id': '4bf58dd8d48988d11f941735', 'name': 'N...",v-1591436704,False,3834 31st St,40.753056,-73.933011,"[{'label': 'display', 'lat': 40.753056, 'lng':...",1645,11101,US,Long Island City,NY,United States,"[3834 31st St, Long Island City, NY 11101, Uni...",NaN,long Island city,501484224,NaN,NaN,NaN,NaN,NaN,NaN
3,57da0f88498e0dee5c0e75ee,Embassy Nightclub & Lounge,"[{'id': '4bf58dd8d48988d11f941735', 'name': 'N...",v-1591436704,False,33-02 Queens Blvd,40.744403,-73.931507,"[{'label': 'display', 'lat': 40.744403, 'lng':...",2004,11101,US,Queens,NY,United States,"[33-02 Queens Blvd (33rd Street), Long Island ...",33rd Street,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,51ac180a498e83fcf1d990ed,Melao Night Club,"[{'id': '4bf58dd8d48988d11f941735', 'name': 'N...",v-1591436704,False,7905 Roosevelt Ave,40.747616,-73.886475,"[{'label': 'display', 'lat': 40.747616, 'lng':...",2536,11372,US,Jackson Heights,NY,United States,"[7905 Roosevelt Ave, Jackson Heights, NY 11372...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now I will filter the Dataframe to only show the information that is important to the client

In [53]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in venues_queens_df.columns if col.startswith('location.')] + ['id']
venues_queens_df_filtered = venues_queens_df.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
venues_queens_df_filtered['categories'] = venues_queens_df_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
venues_queens_df_filtered.columns = [column.split('.')[-1] for column in venues_queens_df_filtered.columns]

venues_queens_df_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,PureNY Nightclub,Nightclub,62-05 30th Ave,40.760051,-73.901250,"[{'label': 'display', 'lat': 40.76005082320337...",1146,11377,US,Woodside,NY,United States,"[62-05 30th Ave, Woodside, NY 11377, United St...",NaN,NaN,4bc92319511f9521eb4aadc7
1,Star Nightclub,Pub,246 Columbus Ave,40.777270,-73.979207,"[{'label': 'display', 'lat': 40.77727, 'lng': ...",5950,10023,US,New York,NY,United States,"[246 Columbus Ave (btw 71st and 72nd), New Yor...",btw 71st and 72nd,NaN,42a39200f964a520ea241fe3
2,Doha Nightclub NYC,Nightclub,3834 31st St,40.753056,-73.933011,"[{'label': 'display', 'lat': 40.753056, 'lng':...",1645,11101,US,Long Island City,NY,United States,"[3834 31st St, Long Island City, NY 11101, Uni...",NaN,long Island city,5a58a0b5c47cf954cad00ad1
3,Embassy Nightclub & Lounge,Nightclub,33-02 Queens Blvd,40.744403,-73.931507,"[{'label': 'display', 'lat': 40.744403, 'lng':...",2004,11101,US,Queens,NY,United States,"[33-02 Queens Blvd (33rd Street), Long Island ...",33rd Street,NaN,57da0f88498e0dee5c0e75ee
4,Melao Night Club,Nightclub,7905 Roosevelt Ave,40.747616,-73.886475,"[{'label': 'display', 'lat': 40.747616, 'lng':...",2536,11372,US,Jackson Heights,NY,United States,"[7905 Roosevelt Ave, Jackson Heights, NY 11372...",NaN,NaN,51ac180a498e83fcf1d990ed
5,Amadeus Nightclub,Nightclub,79-51 Albion Ave,40.739398,-73.884421,"[{'label': 'display', 'lat': 40.73939815942513...",3153,11373,US,Elmhurst,NY,United States,"[79-51 Albion Ave, Elmhurst, NY 11373, United ...",NaN,NaN,56609d3b498efe11e6679c4f
6,La Boom,Nightclub,5615 Northern Blvd,40.753597,-73.904453,"[{'label': 'display', 'lat': 40.75359733615381...",880,11377,US,Woodside,NY,United States,"[5615 Northern Blvd (at 57th St), Woodside, NY...",at 57th St,NaN,4b65438ff964a52013eb2ae3
7,LIT NYC GENTLEMEN CLUB,Nightclub,35-35 Steinway St,40.754276,-73.922647,"[{'label': 'display', 'lat': 40.75427616575544...",768,11101,US,Astoria,NY,United States,"[35-35 Steinway St (Steinway street), Astoria,...",Steinway street,NaN,528276f211d215cb2b1f7e1f
8,Medusa Night Club,Nightclub,25-18 50th St,40.763749,-73.904286,"[{'label': 'display', 'lat': 40.76374935949807...",1143,11377,US,Woodside,NY,United States,"[25-18 50th St (25th avenue), Woodside, NY 113...",25th avenue,NaN,53118e23498e874d47549229
9,Pa'l Karajo,Nightclub,6207 Northern Blvd.,40.754076,-73.899922,"[{'label': 'display', 'lat': 40.75407611068678...",1228,11377,US,Jackson Heights,NY,United States,"[6207 Northern Blvd., Jackson Heights, NY 1137...",NaN,NaN,4e1958dad1648b83483b7adf


In [55]:
venues_queens_df_filtered.count()

name                24
categories          24
address             20
lat                 24
lng                 24
labeledLatLngs      24
distance            24
postalCode          21
cc                  24
city                23
state               24
country             24
formattedAddress    24
crossStreet         13
neighborhood         1
id                  24
dtype: int64

 ### We can observe that there are only 24 Nightclub in this area. 

From our results we can conclude that Manhattan would be a better area for our client to move to rather than Queens.